Programa Principal que descarga información de las Companías desde la Superintendencia de Companías

In [203]:
# instalar en un cmd pip install matplotlib
# pip install openpyxl


from selenium.webdriver.chrome.options import Options
from time import sleep
import pandas as pd
import numpy as np
from IPython.core.display import HTML
import matplotlib.pyplot as plt
import os
from datetime import datetime

#object of Options class
opciones = webdriver.ChromeOptions()

#opciones del navegador
options.add_argument("--start-maximized")
prefs = {"profile.default_content_settings.popups": 0,
             "download.default_directory":
                        r'Ranking Empresas',
             "directory_upgrade": True}
opciones.add_experimental_option("prefs", prefs)

#hora local para adicionar al nombre del archivo descargado
local_dt = str(datetime.now())
local_dt_str=local_dt.replace(' ','').replace(':','').replace('.','')

#set chromedriver.exe path
driver = webdriver.Chrome(executable_path=r"chromedriver.exe",
options=opciones)

##launch URL
driver.get("https://appscvs.supercias.gob.ec/rankingCias/")

Rankings=[]

for i in ["ESMERALDAS","MANABÍ","SANTA ELENA"] :#,"MANABÍ","SANTA ELENA","GUAYAS","EL ORO", "LOS RÍOS","CARCHI","IMBABURA","PICHINCHA","COTPAXI",
         #"BOLIVAR","CAÑAR","CHIMBORAZO","AZUAY","LOJA","STO. DOMINGO DE LOS TSÁCHILAS","TUNGURAHUA","ORELLANA",
         #"NAPO","SUCUMBIOS","MORONA SANTIAGO","ZAMORA CHINCHIPE","PASTAZA","GALÁPAGOS"] :
    sleep(5)
    #click en area de provincia
    driver.find_element_by_xpath("//area[@title='%s']"% str(i)).click()
   
    #seteo de opciones de descarga de archivo
    combos=driver.find_elements_by_xpath('//input[contains(@class,"z-combobox-inp")]')
    combos[2].send_keys('Descargar Todas las CIAS')
    combos[3].send_keys('EXCEL')
   
    #clic botón de descarga
    driver.find_element_by_xpath("//img[@src='/rankingCias/images/download.png']/parent::td").click()
    sleep(6)
   
    #cambio nombre de archivo descargado
    os.rename(r'C:\Users\luisp\Downloads\ReporteRanking.xlsx',r'C:\Users\luisp\Downloads\ReporteRanking'+str(i)+str(local_dt_str)+'.xlsx')
   
    #busqueda de valores de cada celda de la tabla de la página
    listaValores=driver.find_elements_by_xpath('//div[contains(@class,"z-listcell-cnt")]')
    valores=[valor.text for valor in listaValores]
   
    #coloca en un dataframe
    df = pd.DataFrame(np.array(valores).reshape(int(len(listaValores)/16),16),
                           columns = ["Ranking_2020","Ranking_2019","Posicion","Actividad","Nombre_Empresa","Provincia","Ciudad",
                                      "Tamanio","Activos","Patrimonio","Venta","Ut_Ant_Imp",
                                      "Ut_Ejercicio","Ut_Neta","Imp_Rent_Causado","Ingreso_Total"])
    #formato de números a las columnas
    df[df.columns[8:]] = df[df.columns[8:]].apply(lambda x: x.str.replace('$','', regex=True)).apply(lambda x: x.str.replace('.','', regex=True)).apply(lambda x: x.str.replace(',','.', regex=True)).astype(float)
    
        
    Rankings.append(df)
    
    #propiedades del gráfico
    #colors = ["#118DFF", "#FFDD01", "#D63B59"]
    #grp=df.groupby(['tamanio']).sum().plot(kind='pie', y='patrimonio',colors=colors,title=str(i),subplots=True, shadow = True,
    #legend=True,autopct='%1.1f%%',startangle=20, explode=(.1,0,.1))
    
    #grp2= df.groupby(['tamanio']).sum().plot(kind='bar', y='activos',colors=colors,title=str(i))
   
    #despliega tabla
    display(HTML(df.to_html()))
   
    #despliega grafico
    plt.show()    
   
    #regresa a la página principal
    driver.back()

    
RankingEmpresas=pd.concat(Rankings,ignore_index=True)
 # guardar dataframe en un excel
file_name = r'C:\Users\luisp\Downloads\RankingEmpresasSuperCompanias-'+str(local_dt_str)+'.xlsx' #ruta donde se guarda el dataframe
RankingEmpresas.to_excel(file_name)
file_name = r'RankingEmpresasSuperCompanias.xlsx' #ruta donde se guarda el dataframe a ser cargado
RankingEmpresas.to_excel(file_name)

,Ranking_2020,Ranking_2019,Posicion,Actividad,Nombre_Empresa,Provincia,Ciudad,Tamanio,Activos,Patrimonio,Venta,Ut_Ant_Imp,Ut_Ejercicio,Ut_Neta,Imp_Rent_Causado,Ingreso_Total
0,44,3374,1,,SUMINISTROS Y MANTENIMIENTOS DE EQUIPOS PETROL...,ESMERALDAS,ESMERALDAS,PEQUEÑA,3355980.30,1013828.51,489014.81,0.00,-1247698.47,0.00,0.00,510702.17
1,926,35731,2,,SERVITECNICAS BANEVALL S.A.,ESMERALDAS,ESMERALDAS,MICROEMPRESA,84676.19,82515.38,40006.70,12090.66,14224.31,12090.66,0.00,40006.70
2,1230,-,3,,"COMPAÑIA DE PRODUCCIÓN, INDUSTRIALIZACIÓN, TRA...",ESMERALDAS,ESMERALDAS,MICROEMPRESA,40000.00,40000.00,0.00,19666.25,-11652.59,19666.25,0.00,19666.25
3,1653,51283,4,,TRANSPORTES NEPTUNO & NEPTUNO TRANSNEPTUNO C.A.,ESMERALDAS,ESMERALDAS,MICROEMPRESA,12949.70,12949.70,89847.97,6242.08,7033.33,5255.65,986.43,89847.97
4,1839,52114,5,,"''FRÍO-RICO'' DISEÑO DE SISTEMA, INSTALACIÓN, ...",ESMERALDAS,ESMERALDAS,MICROEMPRESA,8370.00,8370.00,12414.30,100.00,100.00,72.00,28.00,12414.30
5,1988,54919,6,,CONSTRUCTORA GRETTA CONSTRUC S.A.,ESMERALDAS,ROSA ZARATE (QUININDÉ),MICROEMPRESA,5040.30,4407.77,0.00,0.00,0.00,0.00,0.00,0.00
6,2063,56071,7,,AGENCIA NAVIERA ANDROMESHIPPING S.A.,ESMERALDAS,ESMERALDAS,,4000.00,4000.00,0.00,0.00,0.00,0.00,0.00,0.00
7,2348,60126,8,,UNION DE TAXIS SAN LORENZO PAILOTAXI S.A.,ESMERALDAS,SAN LORENZO,PEQUEÑA,1589.42,1589.42,0.00,0.00,0.00,0.00,0.00,0.00
8,2357,60328,9,,COMPAÑIA DE TRANSPORTE DE CAMIONETAS MIXTA DOB...,ESMERALDAS,ESMERALDAS,MICROEMPRESA,1521.57,1500.00,424.00,54.40,64.00,42.43,11.97,424.00
9,2359,60474,10,,COMPAÑIA DE TRANSPORTE EN TRICIMOTO UNION DE T...,ESMERALDAS,ATACAMES,MICROEMPRESA,1500.00,1000.00,1475.00,83.65,83.65,62.74,20.91,1475.00


,Ranking_2020,Ranking_2019,Posicion,Actividad,Nombre_Empresa,Provincia,Ciudad,Tamanio,Activos,Patrimonio,Venta,Ut_Ant_Imp,Ut_Ejercicio,Ut_Neta,Imp_Rent_Causado,Ingreso_Total
0,10,2188,1,,"MANACRIPEX CIA. LTDA. MANABITA DE CRIA , PESCA...",MANABI,MANTA,MEDIANA,9469487.88,5811415.67,4567517.75,160996.51,189407.66,52805.30,108191.21,4655777.04
1,93,9463,2,,HORMIGONES DEL PACIFICO ''HORDEPAC'' S.A,MANABI,JARAMIJÓ,MEDIANA,1271369.18,639315.87,1169439.29,5955.71,7006.72,4466.78,1488.93,1169546.07
2,106,10585,3,,HIELASO S.A.,MANABI,PEDERNALES,PEQUEÑA,1094087.20,622956.02,958458.75,1180.85,1389.24,130.58,1050.27,960894.46
3,119,60321,4,,SAIMBIN S.A.,MANABI,MANTA,MICROEMPRESA,912645.33,1091.71,47560.83,91.71,91.71,71.53,20.18,47560.83
4,155,13251,5,,COMPAÑIA DE TRANSPORTE DE CARGA PESADA TRUCKVE...,MANABI,JUNÍN,PEQUEÑA,632284.55,329294.99,178263.14,0.55,-562.09,0.00,880.47,178263.69
5,174,15307,6,,DISTRIBUIDORAROGUIZAM CIA.LTDA.,MANABI,MANTA,MEDIANA,499328.33,50597.89,2258897.82,13440.32,15812.14,10080.24,3360.08,2258917.83
6,197,15280,7,,INMOTUNA S.A.,MANABI,MANTA,MICROEMPRESA,494414.33,286705.09,0.00,0.00,-1281.56,0.00,0.00,0.00
7,345,24718,8,,MARMONTESAN S.A.,MANABI,MONTECRISTI,PEQUEÑA,346319.58,24641.82,506095.02,159360.35,3216.29,158583.62,776.73,665937.81
8,363,18883,9,,OEE DISTRIBUCIONES S.A.,MANABI,JARAMIJÓ,PEQUEÑA,327453.83,252250.07,458064.94,7336.65,646.94,7215.67,120.98,466170.29
9,387,50863,10,,DRINI S.A.,MANABI,PORTOVIEJO,MICROEMPRESA,310223.27,162192.69,65137.37,24302.53,28591.21,18955.97,5346.56,65137.37


,Ranking_2020,Ranking_2019,Posicion,Actividad,Nombre_Empresa,Provincia,Ciudad,Tamanio,Activos,Patrimonio,Venta,Ut_Ant_Imp,Ut_Ejercicio,Ut_Neta,Imp_Rent_Causado,Ingreso_Total
0,75,6374,1,,BRUMESA S.A.,SANTA ELENA,SANTA ELENA,MEDIANA,1659992.91,253782.88,4176723.42,0.00,-60818.15,0.00,0.00,4187574.56
1,239,16293,2,,BARZOLA CHAVEZ FERRETERIA S.A. BACHAFESA,SANTA ELENA,LA LIBERTAD,PEQUEÑA,460226.14,37088.07,626176.64,0.00,-39329.70,0.00,0.00,626176.64
2,297,19950,3,,AYANGUE PACIFIC TURIS APT S.A.,SANTA ELENA,COLONCHE,PEQUEÑA,396885.08,304240.38,176800.00,-1108.56,7330.43,-1108.56,0.00,211455.47
3,311,17848,4,,CONSTRUDISMAT S.A.,SANTA ELENA,SALINAS,PEQUEÑA,381421.89,50348.65,463310.36,0.00,-5276.97,0.00,0.00,463310.36
4,415,17313,5,,AQUALAB SA,SANTA ELENA,SANTA ELENA,MICROEMPRESA,292114.73,120000.00,0.00,0.00,0.00,0.00,0.00,0.00
5,440,24926,6,,"INMOSEG, INMOBILIARIA SEGUNDA S.A.",SANTA ELENA,SALINAS,PEQUEÑA,275035.89,275019.24,0.00,200538.27,159233.64,200538.27,0.00,200538.27
6,464,23258,7,,PACIF.LIB S.A.,SANTA ELENA,LA LIBERTAD,MICROEMPRESA,259857.37,258799.93,70072.32,10111.11,10111.11,7583.33,2527.78,70072.32
7,564,24714,8,,OBRAS CIVILES Y CONSTRUCCIONES SALAV S.A. CONS...,SANTA ELENA,SANTA ELENA,MICROEMPRESA,208065.70,129144.93,86744.29,0.00,-7068.84,0.00,0.00,86744.29
8,613,26419,9,,ECOLARVAS S.A.,SANTA ELENA,SANTA ELENA,PEQUEÑA,181339.28,107039.58,392757.97,0.00,-15888.27,0.00,0.00,392757.97
9,626,24540,10,,RAIZCONSULTORA S.A.,SANTA ELENA,LA LIBERTAD,MICROEMPRESA,171996.08,15412.36,54422.09,264.34,310.99,206.19,58.15,54422.09


In [204]:
RankingEmpresas.to_excel(file_name)